In [7]:
import keras
from keras.layers import Input, Conv2D, Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, CSVLogger, LambdaCallback
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from matplotlib import pyplot as plt
import keras.backend as K
import pandas as pd
import os
import numpy as np
import shutil
from PIL import Image
batch_size = 4
epochs = 400

In [8]:
df=pd.read_csv(r"placesData.csv")
datagen = ImageDataGenerator(rescale=1./255)
seed = 42
image_generator = datagen.flow_from_dataframe(df, 
                                      directory='.', 
                                      x_col='x_train', 
                                      y_col='y_train', 
                                      batch_size=batch_size,
                                      target_size=(512, 512),
                                      class_mode=None, 
                                      seed=seed)
mask_generator = datagen.flow_from_dataframe(df, 
                                     directory='.', 
                                     x_col='y_train', 
                                     y_col='x_train', # Or whatever 
                                     batch_size=batch_size,
                                     target_size=(512, 512),
                                     class_mode=None, 
                                     seed=seed)

train_generator = zip(image_generator, mask_generator)

Found 4000 validated image filenames.
Found 4000 validated image filenames.


In [9]:
input = Input(shape=(512, 512, 3,), batch_shape=(batch_size, 512, 512, 3))
l1 = Conv2D(filters=256, kernel_size=(16, 16), padding="same", activation='elu')(input)
print(l1.shape)
l2 = Conv2D(filters=512, kernel_size=(8, 8), padding="same", activation='elu')(l1)
print(l2.shape)
l3 = Conv2D(filters=512, kernel_size=(1, 1), padding="same", activation='elu')(l2)
print(l3.shape)
output = Conv2D(filters=3, kernel_size=(8, 8), padding="same", activation='sigmoid')(l3)
print(output.shape)
model = Model(inputs=[input], outputs=output)

(4, 512, 512, 256)
(4, 512, 512, 512)
(4, 512, 512, 512)
(4, 512, 512, 3)


In [10]:
model.load_weights("FGNweightsNM.hdf5")

In [11]:
img = image.load_img("places/s01805.png", grayscale=False, target_size=(512, 512))
x = np.array(image.img_to_array(img))
print(x)
x = x[None, :]/255.
x = np.tile(x, (batch_size,1,1,1))
print(x.shape)

[[[ 83.  82.  62.]
  [ 83.  82.  62.]
  [ 83.  82.  62.]
  ...
  [255. 255. 250.]
  [255. 255. 250.]
  [255. 255. 250.]]

 [[ 83.  82.  62.]
  [ 83.  82.  62.]
  [ 83.  82.  62.]
  ...
  [255. 255. 253.]
  [255. 255. 253.]
  [255. 255. 253.]]

 [[ 83.  82.  62.]
  [ 83.  82.  62.]
  [ 83.  82.  62.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[ 34.  35.  29.]
  [ 26.  27.  21.]
  [ 21.  22.  16.]
  ...
  [ 49.  69.  68.]
  [ 49.  69.  68.]
  [ 48.  68.  67.]]

 [[ 24.  25.  19.]
  [ 19.  20.  14.]
  [ 16.  17.  11.]
  ...
  [ 50.  70.  69.]
  [ 49.  69.  68.]
  [ 48.  68.  67.]]

 [[ 17.  18.  12.]
  [ 18.  19.  13.]
  [ 19.  20.  14.]
  ...
  [ 51.  71.  70.]
  [ 50.  70.  69.]
  [ 50.  70.  69.]]]
(4, 512, 512, 3)


In [12]:
alg = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.99999, amsgrad=True)
model.compile(optimizer=alg,
             loss='mean_squared_error',
             metrics=['accuracy', 'mse'])

In [ ]:
checkpoint = ModelCheckpoint("FGNchkptNM.hdf5", monitor='loss', save_best_only=False, mode='auto', period=1, verbose=1)
csv_logger = CSVLogger('training.log')
image_saver = LambdaCallback(on_epoch_end=lambda epoch,logs: Image.fromarray((model.predict(x)[0]*255.).astype('uint8')).save("eImgs/"+str(epoch)+"p.jpeg"))
sanity_check1 = LambdaCallback(on_epoch_end=lambda epoch,logs: print((model.predict(x)[0]*255.)))
model.fit_generator(train_generator, steps_per_epoch=400/batch_size, epochs=epochs, shuffle=True, callbacks=[checkpoint, csv_logger, image_saver], verbose=1)
model.save('FGNweightsNM.hdf5')

Epoch 1/400
100/100 [==============================] - 288s 3s/step - loss: 0.0073 - accuracy: 0.7937 - mse: 0.0073

Epoch 00001: saving model to FGNchkptNM.hdf5
Epoch 2/400
100/100 [==============================] - 288s 3s/step - loss: 0.0070 - accuracy: 0.8248 - mse: 0.0070

Epoch 00002: saving model to FGNchkptNM.hdf5
Epoch 3/400
100/100 [==============================] - 288s 3s/step - loss: 0.0068 - accuracy: 0.8499 - mse: 0.0068

Epoch 00004: saving model to FGNchkptNM.hdf5
Epoch 5/400
100/100 [==============================] - 287s 3s/step - loss: 0.0070 - accuracy: 0.8244 - mse: 0.0070

Epoch 00005: saving model to FGNchkptNM.hdf5
Epoch 6/400
100/100 [==============================] - 288s 3s/step - loss: 0.0069 - accuracy: 0.8441 - mse: 0.0069

Epoch 00006: saving model to FGNchkptNM.hdf5
Epoch 7/400
100/100 [==============================] - 287s 3s/step - loss: 0.2017 - accuracy: 0.5715 - mse: 0.2017

Epoch 00007: saving model to FGNchkptNM.hdf5
Epoch 8/400
100/100 [=======

In [ ]:
model.load_weights("FGNchkptE273Sig.hdf5")

In [ ]:
def save_image(model, path, tgtDir):
    img = image.load_img(path, grayscale=False, target_size=(512, 512))
    x = np.array(image.img_to_array(img))
    x = x[None, :]/255.
    x = np.tile(x, (batch_size,1,1,1))
    Image.fromarray((model.predict(x)[0]*255.).astype('uint8')).save(tgtDir+"/t"+path.replace('/', ' '))
save_image(model, "cac2-distorted.png", "test")

In [ ]:
import glob
for imgPath in glob.glob("places/*"):
    save_image(model, imgPath, "test")

In [ ]:
print(image_generator.next()[0][0][255])
print(mask_generator.next()[0][0][255])

In [ ]:
img = image.load_img("placesMongrels/mongrolized_s00001_1.png", grayscale=False, target_size=(512, 512))
x = np.array(image.img_to_array(img))
img2 = image.load_img("placesMongrels/mongrolized_s00001_2.png", grayscale=False, target_size=(512, 512))
y = np.array(image.img_to_array(img2))
z = np.mean([x, y], axis=0)/255.
z = z[None:]
print(z.shape)
Image.fromarray(z*255).astype('uint8').save("t.png")